In [1]:
import numpy as np

# Read vogs in both groups (viruses, phages)

In [2]:
fr = open("vogsInPhagesAndViruses.txt")
both = []
for line in fr:
    both.append(int(line.strip('\n')))
print len(both)

6304


# Read taxonomy annotation File

In [3]:
fr = open("/home/leonardo/Documents/Projects/VOGss/7_DBtests/iv_queries/taxonomyAnotation.csv")
accessions = {}
for line in fr:
    line = line.strip('\n').split(',')
    accession = line[0]
    taxPath = line[1:] #taxPath means taxonomy path
    accessions[accession] = taxPath
print len(accessions)

#A dictionary of taxonomic labels allowed. 
taxLabels = {"subspecies":0,"species":1,"subgenus":2,"genus":3,"subfamily":4,"family":5,"order":6,"type":7}

13999


### Complete NAs in taxonomy

In [4]:
for accession in accessions:
    annotation = accessions[accession]
    for i in range(8):
        if annotation[i] == 'NA' and i < 7:
            annotation[i] = "NA" + "_" + annotation[i+1]
    accessions[accession] = annotation    
print len(accessions)

13999


# Read VOGs file (by VOG)

In [5]:
fr = open("genomesVsClusters.sortedByCluster.greaterThan2.tab")
clusters = {}

for line in fr:
    line = line.strip('\n').split('\t')
    genome = line[0].split('.')[0]
    cluster = int(line[1].split('CLS')[1])
    if cluster in clusters:
        genomes = clusters[cluster]
        if genome in genomes:
            genomes[genome] = genomes[genome] + 1
        else:
            genomes[genome] = 1
        clusters[cluster] = genomes
    else:
        genomes = {}
        genomes[genome] = 1
        clusters[cluster] = genomes
fr.close()

In [18]:
vogsTaxonomy = {}
allTaxLabels = set([])
for vog in both:
    for genome in clusters[vog]:
        if vog in vogsTaxonomy:
            vogsTaxonomy[vog] = vogsTaxonomy[vog].union([accessions[genome][taxLabels["family"]]]) 
        else:
            vogsTaxonomy[vog] = set([accessions[genome][taxLabels["family"]]])
        allTaxLabels = allTaxLabels.union([accessions[genome][taxLabels["family"]]])
allTaxLabels = list(allTaxLabels)

In [19]:
taxVOGsBoth = np.zeros((len(both),len(allTaxLabels)))
i=0
for vog in vogsTaxonomy:
    for tax in vogsTaxonomy[vog]:
        taxVOGsBoth[i][allTaxLabels.index(tax)] += 1
    i += 1

fw = open("taxVOGsBothFamily.tab","w")
fw.write("x\t")
for tax in allTaxLabels:
    fw.write(tax + '\t')
fw.write('\n')
for i in range(len(vogsTaxonomy.keys())):
    fw.write(str(vogsTaxonomy.keys()[i]) + '\t')
    for value in taxVOGsBoth[i]:
        fw.write(str(value) + '\t')
    fw.write('\n')
fw.close()

### sed ':a;N;$!ba;s/\t\n/\n/g' taxVOGsBothOrder.tab > taxVOGsBothOrder.tab.mod
### modifica el archivo de salida para eliminar el 'ultimo tab

In [21]:
len(allTaxLabels)

68